<a href="https://colab.research.google.com/github/keiverdatascience/labo2025v/blob/main/src/ensembles/420_ArbolesAzarosos_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4. Ensembles de Arboles de Decision

Un arbol de decisión es un modelo débil, el aumento del poder predictivo proviene al ensamblar varios arboles de decisión.
<br> Si promedio n arboles identicos, el resultados es exactamente el mismo que utilizar un solo arbol, necesito PERTURBAR cada arbol para disponer de variablidad

la variabilidad provendrá de estas fuentes:


*   Perturbar el dataset
*   Perturbar el algoritmo del arbol
*   Perturbar el dataset y el algoritmo del arbol al mismo tiempo

Se verán estos tres algoritmos


*   Arboles Azarosos
*   Random Forest
*   Gradient Boosting of Decision Trees

#### 4.01 Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [2]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [3]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi



---



## 4.02 Arboles Azarosos

Arboles Azarosos es el nombre de un algoritmo trivial (por favor NO confundir con Random Forest)
Qué tipo de perturbaciones se realizan en Arboles Azarosos
* Se perturba el dataset
* No se perturba el algoritmo, es siempre rpart original

Cada  arbolito de  Arboles Azarosos se entrena sobre un dataset perturbado,  que tiene exactamente la misma cantidad de registros pero solo un subconjunto de los atributos (campos)  del dataset, tomados al azar, de los originales.
<br> En esta primera corrida, se construira cada arbol en un dataset utilizando el 50% de los campos

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [1]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Tue Oct 28 01:20:00 AM 2025"

In [3]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,727068,38.9,1454468,77.7,1454468,77.7
Vcells,1342651,10.3,8388608,64.0,2167094,16.6


In [15]:
install.packages("rBayesianOptimization")


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘iterators’, ‘lhs’, ‘foreach’, ‘GPfit’




In [16]:
# cargo las librerias que necesito
require("data.table")
require("rpart")
require("rBayesianOptimization")

Loading required package: rBayesianOptimization



Aqui debe cargar SU semilla primigenia

In [5]:
PARAM <- list()
PARAM$semilla_primigenia <- 245681

# parametros  arbol
# entreno cada arbol con solo 50% de las variables variables
#  por ahora, es fijo
PARAM$feature_fraction <- 0.5

# voy a generar 512 arboles,
#  a mas arboles mas tiempo de proceso y MEJOR MODELO,
#  pero ganancias marginales
PARAM$num_trees_max <- 32

In [6]:
# carpeta de trabajo
setwd("/content/buckets/b1/exp")
experimento <- "exp4030"
dir.create(experimento, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento ))

In [7]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv")

In [8]:
# defino los dataset de entrenamiento y aplicacion
dtrain <- dataset[foto_mes == 202107]
dfuture <- dataset[foto_mes == 202109]

# arreglo clase_ternaria por algun distraido ""
dfuture[, clase_ternaria := NA ]

In [9]:
# Establezco cuales son los campos que puedo usar para la prediccion
# el copy() es por la Lazy Evaluation
campos_buenos <- copy(setdiff(colnames(dtrain), c("clase_ternaria")))

In [ ]:
# Función de evaluación (objetivo para la optimización bayesiana)
#   Maximiza la ganancia o la precisión (puedes ajustar a tu métrica)

score_function <- function(cp, minsplit, minbucket, maxdepth) {
  ganancia_total <- 0

  # Simulamos un ensemble pequeño dentro de la evaluación
  for (arbolito in 1:10) {   # pequeño ensemble para estimar rendimiento
    qty_campos <- as.integer(length(campos_buenos) * PARAM$feature_fraction)
    campos_random <- sample(campos_buenos, qty_campos)
    formulita <- paste0("clase_ternaria ~ ", paste(campos_random, collapse=" + "))

    modelo <- rpart(
      formulita,
      data = dtrain,
      xval = 0,
      control = rpart.control(
        cp = cp,
        minsplit = as.integer(minsplit),
        minbucket = as.integer(minbucket),
        maxdepth = as.integer(maxdepth)
      )
    )

    pred <- predict(modelo, dtrain, type="prob")[, "BAJA+2"]

    # Ejemplo de métrica: ganancia esperada (ajústala según tu negocio)
    ganancia <- sum((pred > 0.025) * ifelse(dtrain$clase_ternaria == "BAJA+2", 117000, -3000))
    ganancia_total <- ganancia_total + ganancia
  }

  ganancia_promedio <- ganancia_total / 10
  return(list(Score = ganancia_promedio, Pred = 0))
}

#----------------------------------
# Búsqueda de hiperparámetros
bounds <- list(
  cp = c(-1, -0.0001),
  minsplit = c(200L, 2000L),
  minbucket = c(50L, 1000L),
  maxdepth = c(4L, 16L)
)

set.seed(PARAM$semilla_primigenia)
opt_result <- BayesianOptimization(
  FUN = score_function,
  bounds = bounds,
  init_points = 10,
  n_iter = 30,
  acq = "ucb",
  kappa = 2.5,
  eps = 0.0,
  verbose = TRUE
)

print(opt_result$Best_Par)
print(paste("Mejor Score:", round(opt_result$Best_Value, 2)))



In [ ]:
# que tamanos de ensemble grabo a disco
grabar <- c(1, 2, 4, 8, 16, 32)

In [ ]:
#----------------------------------
# Entrenar ensemble final con los mejores hiperparámetros
best_cp <- opt_result$Best_Par["cp"]
best_minsplit <- opt_result$Best_Par["minsplit"]
best_minbucket <- opt_result$Best_Par["minbucket"]
best_maxdepth <- opt_result$Best_Par["maxdepth"]

tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob_acumulada := 0]

for (arbolito in seq(PARAM$num_trees_max)) {
  message("Entrenando árbol ", arbolito)

  qty_campos <- as.integer(length(campos_buenos) * PARAM$feature_fraction)
  campos_random <- sample(campos_buenos, qty_campos)
  formulita <- paste0("clase_ternaria ~ ", paste(campos_random, collapse=" + "))

  modelo <- rpart(
    formulita,
    data = dtrain,
    xval = 0,
    control = rpart.control(
      cp = best_cp,
      minsplit = as.integer(best_minsplit),
      minbucket = as.integer(best_minbucket),
      maxdepth = as.integer(best_maxdepth)
    )
  )

  prediccion <- predict(modelo, dfuture, type="prob")
  tb_prediccion[, prob_acumulada := prob_acumulada + prediccion[, "BAJA+2"]]

  if (arbolito %in% grabar) {
    umbral_corte <- (1 / 40) * arbolito
    tb_prediccion[, Predicted := as.numeric(prob_acumulada > umbral_corte)]

    archivo_kaggle <- paste0(
        "KA420_",
        sprintf("%.3d", arbolito), # para que tenga ceros adelante
        ".csv"
      )

    # grabo el archivo
    fwrite( tb_prediccion[, list(numero_de_cliente, Predicted)],
      file= archivo_kaggle,
      sep= ","
    )

        # subida a Kaggle
    comando <- "kaggle competitions submit"
    competencia <- "-c   labo-i-2025-virtual-analista-sr"
    arch <- paste( "-f", archivo_kaggle)


    mensaje <- paste0("-m 'cp=", PARAM$rpart$cp, "  minsplit=", PARAM$rpart$minsplit, "  minbucket=", PARAM$rpart$minbucket, " maxdepth=", PARAM$rpart$maxdepth, "'" )
    linea <- paste( comando, competencia, arch, mensaje)
    salida <- system(linea, intern=TRUE)
    cat(salida)
}
}


In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Wed Oct 22 08:34:36 AM 2025"

[1] "Wed Oct 22 08:34:36 AM 2025"



---

